In [1]:
import mysql.connector as sql
import pandas as pd
import datetime as dt
import numpy as np
from sorted_months_weekdays import *
from sort_dataframeby_monthorweek import *
import matplotlib.pyplot as plt
import math
import requests
import json
from pandas.io.json import json_normalize
import matplotlib.dates as mdates
from datetime import date, timedelta

In [2]:
db_connection = sql.connect(host='localhost', database='collection_users', user='pandas', password='password')

sbBundles = pd.read_sql('SELECT * FROM UsersShoeboxBundles', con=db_connection)
sbBundles['created'] = pd.to_datetime(sbBundles['created'], unit='s')
sbBundles['lastmodified'] = pd.to_datetime(sbBundles['lastmodified'], unit='s')
sbBundles['deleted'].replace(0, np.nan, inplace=True)
sbBundles['deleted'] = pd.to_datetime(sbBundles['deleted'], unit='s')
sbBundles['description'].replace(r'^\s*$', np.NaN, regex=True, inplace=True)

sbItems = pd.read_sql('SELECT * FROM UsersShoeboxItems', con=db_connection)
sbItems['tool_id'] = sbItems['tool_id'].astype('category')
sbItems['extension'].replace(r'^\s*$', np.NaN, regex=True, inplace=True)
sbItems['extension'] = sbItems['extension'].astype('category')
sbItems['mimetype'] = sbItems['mimetype'].astype('category')
sbItems['created'] = pd.to_datetime(sbItems['created'], unit='s')
sbItems['lastmodified'] = pd.to_datetime(sbItems['lastmodified'], unit='s')
sbItems['deleted'].replace(0, np.nan, inplace=True)
sbItems['deleted'] = pd.to_datetime(sbItems['deleted'], unit='s')
sbItems['refers_to_uid'] = sbItems['refers_to_uid'].astype(int)
sbItems['fingerprint'].replace(r'^\s*$', np.NaN, regex=True, inplace=True)
sbItems['title'].replace(r'^\s*$', np.NaN, regex=True, inplace=True)
sbItems['description'].replace(r'^\s*$', np.NaN, regex=True, inplace=True)
sbItems['secret'].replace(r'^\s*$', np.NaN, regex=True, inplace=True)
sbItems['secret_o'].replace(r'^\s*$', np.NaN, regex=True, inplace=True)
sbItems['extras'].replace(r'^\s*$', np.NaN, regex=True, inplace=True)

In [5]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 50)

In [6]:
sbItemsNan = sbItems[~pd.isnull(sbItems['secret'])]
sbItemsNan

,id,user_id,action_id,tool_id,bundle_id,refers_to_id,refers_to_uid,fingerprint,title,description,secret,extension,mimetype,is_public,created,lastmodified,deleted,secret_o,extras,status_id,processed,input_id
0,789443,199,2,0,789383,0,0,e6c8ee7e3c4d75c6292befc26c24dc884db71cd4,NaN,NaN,UqbTQrnbo5BV,png,image/png,0,2014-09-30 20:16:24,2014-09-30 20:38:58,NaT,UqbTQrnbo5BV,NaN,0,0,0
1,789513,199,2,0,789383,0,0,19260472a130fd3dce2671a3233cedf207729992,NaN,NaN,ARfJNtmR27n6,svg,image/svg+xml,0,2014-09-30 20:40:41,2014-09-30 20:40:41,NaT,ARfJNtmR27n6,NaN,0,0,0
7,790373,247,2,8,790365,1,18731913,e612768311e890008508d5feb8cbdc2fda3167c4,NaN,NaN,qCt9zLY122sJ,obj,application/octet-stream,0,2014-10-06 19:34:33,2015-01-31 04:17:29,2015-01-31 04:17:29,qCt9zLY122sJ,NaN,0,0,0
14,790431,247,2,8,790365,1,18402775,4f23b388bee89529dcbe4440ce4c30cc64786cbe,NaN,NaN,UvMiW2DqKt73,jpg,image/jpeg,0,2014-10-06 20:41:13,2015-01-31 04:17:30,2015-01-31 04:17:30,UvMiW2DqKt73,NaN,0,0,0
15,790453,247,2,3,790365,1,18622423,c70d0985ff981241922a2e8dfc4bc90107135de2,NaN,NaN,XZZYrRBy6oH2,jpg,image/jpeg,0,2014-10-06 21:09:43,2015-01-31 04:17:30,2015-01-31 04:17:30,XZZYrRBy6oH2,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1551218,132873405,28814,2,12,132777661,0,1595449503,33c9597495062da473d5549236cd2fc2e20c2b90,My Table,Made at Cooper Hewitt Museum using a Collectio...,1r95y5TPm68i,stl,application/sla,1,2020-07-22 20:25:03,2020-07-22 20:27:15,NaT,uLav43JZmGmU,"{""fingerprints"":{""o"":{""sha1"":""81a8e64e80ae2e39...",0,0,3
1551219,132873409,28814,2,12,132777661,0,1595449610,a551a0c167d311dca8870d17c1ef667a9427a3dc,My Table,Made at Cooper Hewitt Museum using a Collectio...,NAgXaWDphJr5,stl,application/sla,1,2020-07-22 20:26:50,2020-07-22 20:27:15,NaT,sh81gVyzzCrV,"{""fingerprints"":{""o"":{""sha1"":""b12d3a36ddf17104...",0,0,3
1551220,132873413,28814,2,12,132777661,0,1595449666,d83bdc00860d5ab0a52d48dbe31355891bb27660,My Table,Made at Cooper Hewitt Museum using a Collectio...,ji80CCzZRmIa,stl,application/sla,1,2020-07-22 20:27:46,2020-07-22 20:28:34,NaT,JcXYNRvmdpxu,"{""fingerprints"":{""o"":{""sha1"":""07f206dd7674bbea...",0,0,3
1551221,132873417,28814,2,3,132777661,0,1595450275,0216ae00bc7b1fb50fbf0d8a5c7c188124c5668b,My Wallpaper,Made at Cooper Hewitt Museum using a Collectio...,l5XD9WnrFyPH,svg,image/svg,1,2020-07-22 20:37:55,2020-07-22 20:38:13,NaT,DdF0DmTZdGo9,"{""fingerprints"":{""o"":{""sha1"":""de7d62a819f021b3...",0,0,3


In [19]:
sbBundles = sbBundles.rename({'id': 'bundle_id'}, axis=1)

In [22]:
bundle_merge = pd.merge(sbBundles, sbItemsNan, on='bundle_id', how='inner', suffixes=('_SBbundle', '_SBitems'))
bundle_merge = bundle_merge.drop(['title_SBbundle', 'description_SBbundle', 'count_items_public', 'refers_to_id', 'exported', 'title_SBitems', 'description_SBitems', 
                  'extension', 'mimetype', 'extras', 'status_id', 'processed', 'input_id', 'is_public_SBbundle', 
                   'fingerprint', 'is_public_SBitems', 'secret_o', 'id'], axis=1)

In [23]:
bundle_merge

,bundle_id,user_id_SBbundle,type_id,created_SBbundle,lastmodified_SBbundle,deleted_SBbundle,count_items,user_id_SBitems,action_id,tool_id,refers_to_uid,secret,created_SBitems,lastmodified_SBitems,deleted_SBitems
0,789383,199,1,2014-09-30 17:24:19,2015-04-01 22:02:20,NaT,3,199,2,0,0,UqbTQrnbo5BV,2014-09-30 20:16:24,2014-09-30 20:38:58,NaT
1,789383,199,1,2014-09-30 17:24:19,2015-04-01 22:02:20,NaT,3,199,2,0,0,ARfJNtmR27n6,2014-09-30 20:40:41,2014-09-30 20:40:41,NaT
2,790365,247,1,2014-10-06 19:34:33,2015-04-01 22:02:20,2015-01-31 04:17:29,0,247,2,8,18731913,qCt9zLY122sJ,2014-10-06 19:34:33,2015-01-31 04:17:29,2015-01-31 04:17:29
3,790365,247,1,2014-10-06 19:34:33,2015-04-01 22:02:20,2015-01-31 04:17:29,0,247,2,8,18402775,UvMiW2DqKt73,2014-10-06 20:41:13,2015-01-31 04:17:30,2015-01-31 04:17:30
4,790365,247,1,2014-10-06 19:34:33,2015-04-01 22:02:20,2015-01-31 04:17:29,0,247,2,3,18622423,XZZYrRBy6oH2,2014-10-06 21:09:43,2015-01-31 04:17:30,2015-01-31 04:17:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22747,132777661,28814,1,2020-07-22 20:26:22,2020-07-22 20:54:50,NaT,5,28814,2,3,1595450275,l5XD9WnrFyPH,2020-07-22 20:37:55,2020-07-22 20:38:13,NaT
22748,132777661,28814,1,2020-07-22 20:26:22,2020-07-22 20:54:50,NaT,5,28814,2,14,1595451288,KjKUhqUIThgv,2020-07-22 20:54:48,2020-07-22 20:54:50,NaT
22749,132794701,28814,1,2020-07-22 20:00:52,2020-07-22 20:06:09,NaT,4,28814,2,16,1595447918,SfZy7AZhn5Qy,2020-07-22 19:58:38,2020-07-22 20:04:07,NaT
22750,132794701,28814,1,2020-07-22 20:00:52,2020-07-22 20:06:09,NaT,4,28814,2,11,1595448191,FHCJKgt4vApS,2020-07-22 20:03:11,2020-07-22 20:04:07,NaT


In [24]:
bundle_merge['bundle_id'].count()
#22752

22752

In [25]:
sbBundles['bundle_id'].count()

23126

In [26]:
bundle_merge['bundle_id'].count() / sbBundles['bundle_id'].count() * 100
#98.4 percentage from sbBundle have secrets ---> does it mean they logged online? 

98.38277263685895